Calculation of Unmixing Coefficients and G Parameter
====================================================
---
_Borys Olifirov, 07.2023_

## Theoretical intro

Based on _Zal and Gascoigne, 2004, doi: 10.1529/biophysj.103.022087_

__Intensity notation__

$$I_{XM(T)}$$

- X - excitation type
- M - emission type
- T - transfection type (A if acceptor only cells), does not present for co-transfected cells


### Sensitized-emission

$$F_c = I_{DA} - a (I_{AA} - c I_{DD}) - d (I_{DD} - b I_{AA})$$

$$F_c = I_{DA} - a I_{AA} - d I_{DD}$$
$$if$$
$$b \approx c \approx 0$$


### Crosstalc caliration

$$a = \frac{I_{DA(A)}}{I_{AA(A)}}$$
$$b = \frac{I_{DD(A)}}{I_{AA(A)}}$$
$$c = \frac{I_{AA(D)}} {I_{DD(D)}}$$
$$d = \frac{I_{DA(D)}} {I_{DD(D)}}$$

$$b \approx c \approx 0$$


### Determination of the G parameter

$$G = \frac{F_c}{I_{DD}^{post} - I_{DD}} = \frac{(I_{DA} - a I_{AA} - d I_{DD}) - (I_{DA}^{post} - a I_{AA}^{post} - d I_{DD}^{post})}{I_{DD}^{post} - I_{DD}}$$

$$\Delta I_{DD} = G \cdot \Delta F_c$$


### E-FRET

$$E_{app} = \frac{R}{R+G}, R = \frac{F_c}{I_{DD}}$$

$$E_{corr} = E_{app}(t) \frac{[A^0]_{tot}}{[A^t]_{tot}} = E_{app}(t) \times \frac{\overline{I_{AA}^0 - c I_{DD}^0}}{\overline{I_{AA}(t) - c I_{DD}(t)}}$$

## Script

In [1]:
import numpy as np
from numpy import ma
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as anm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import colors
from matplotlib.colors import LinearSegmentedColormap

import plotly.express as px

from skimage.util import montage
from skimage.filters import rank
from skimage import morphology
from skimage import exposure
from skimage import measure
from skimage import filters
from skimage import io
from skimage import transform
from skimage import registration

from scipy import ndimage
from scipy import signal
from scipy import stats
from scipy import ndimage as ndi

from domb.fret import coef_calc

### Crosstalk

#### Data uploading

In [2]:
img_path = 'data/fret_calibration/crosstalk/'

donor_reg = {'01':[400, 200],  # [exp 435 ms, exp 505 ms]
             '02':[700, 300],
             '03':[500, 400]}
acceptor_reg = {'04':[500, 100],
                '05':[700, 200],
                '06':[500, 300]}

In [3]:
calib_reg = coef_calc.CrossRegSet(data_path=img_path,
                             donor_reg_dict = donor_reg,
                             acceptor_reg_dict=acceptor_reg)

In [4]:
abcd_df = calib_reg.get_abcd()
abcd_df

,ID,type,A_exp,D_exp,coef,val,sd
0,04,A,100,500,a,0.118540,0.191869
1,04,A,100,500,b,0.102553,0.097014
2,05,A,200,700,a,-3.391139,6.171030
3,05,A,200,700,b,0.079326,0.085092
4,06,A,300,500,a,0.044715,0.011843
5,06,A,300,500,b,0.100661,0.067001
6,01,D,200,400,c,0.015066,0.068368
7,01,D,200,400,d,1.146896,1.675272
8,02,D,300,700,c,0.000152,0.029394
9,02,D,300,700,d,0.770531,0.206369


In [8]:
calib_reg.cross_raw_df

,ID,type,A_exp,D_exp,frame,coef,val
0,04,A,100,500,0,a,0.156314
1,04,A,100,500,1,a,0.145575
2,04,A,100,500,2,a,0.121895
3,04,A,100,500,3,a,0.131954
4,04,A,100,500,4,a,0.108104
...,...,...,...,...,...,...,...
583,03,D,400,500,44,d,0.926637
584,03,D,400,500,45,d,0.859968
585,03,D,400,500,46,d,0.923478
586,03,D,400,500,47,d,0.866721


In [5]:
fig = px.box(calib_reg.cross_raw_df, x='coef', y='val', color='ID', points='all')
fig.show()

In [7]:
a_mean = np.mean(abcd_df['val'][(abcd_df['coef'] == 'a') & (abcd_df['ID'] != '05')])
a_sd = np.std(abcd_df['val'][(abcd_df['coef'] == 'a') & (abcd_df['ID'] != '05')])
print(f'a={round(a_mean, 4)}+/-{round(a_sd, 4)}')

b_mean = np.mean(abcd_df['val'][(abcd_df['coef'] == 'b') & (abcd_df['ID'] != '05')])
b_sd = np.std(abcd_df['val'][(abcd_df['coef'] == 'b') & (abcd_df['ID'] != '05')])
print(f'b={round(b_mean, 4)}+/-{round(b_sd, 4)}')

c_mean = np.mean(abcd_df['val'][(abcd_df['coef'] == 'c') & (abcd_df['ID'] != '03')])
c_sd = np.std(abcd_df['val'][(abcd_df['coef'] == 'c') & (abcd_df['ID'] != '03')])
print(f'c={round(c_mean, 4)}+/-{round(c_sd, 4)}')

d_mean = np.mean(abcd_df['val'][abcd_df['coef'] == 'd'])
d_sd = np.std(abcd_df['val'][abcd_df['coef'] == 'd'])
print(f'd={round(d_mean, 4)}+/-{round(d_sd, 4)}')

a=0.0816+/-0.0369
b=0.1016+/-0.0009
c=0.0076+/-0.0075
d=0.9769+/-0.1558


Concatenation of all coeficients data frames

__a__=0.144+/-0.0318

__b__=0.1292+/-0.0027

__c__=0.2458+/-0.0298

__d__=0.8506+/-0.0138


Vis.

### G parameter

#### Parameters

In [ ]:
a = 0.144
b = 0.1292
c = 0.2458
d = 0.8506
abcd = [a, b, c, d]

tandem_path = 'data/fret_calibration/tandem/'

G_fov = coef_calc.GReg(img_name='FOV2',
                        raw_path=tandem_path + '03.tif',
                        bleach_path=tandem_path + '04.tif',
                        bleach_frame=100,
                        bleach_exp=800,
                        A_exp=100,
                        D_exp=400,
                        coef_list=abcd)

G_fov.ch_pic()
G_fov.Fc_DD_pic()
G_fov.G_calc_fit(bad_cells=[3])
G_fov.Fc_plot()